<a href="https://colab.research.google.com/github/bj-bestjung/Onjira-Portfolio/blob/main/US_Stock_Alert_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install yfinance pandas numpy matplotlib plotly ta torch pytorch-forecasting lightning requests

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 55.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 44.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.9/260.9 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 821.1/821.1 kB 31.2 MB/s eta 0:00:00


In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import ta
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from datetime import datetime

In [ ]:
class SimpleStockPredictor:
    def __init__(self):
        # หุ้นที่จะวิเคราะห์
        self.stocks = ['AAPL', 'TSLA', 'MSFT', 'NVDA', 'AMZN']
        self.data = {}
        self.predictions = {}

    def get_stock_data(self, symbol, days=365):
        """ดึงข้อมูลหุ้น"""
        print(f"📥 กำลังดึงข้อมูล {symbol}...")

        try:
            # ดึงข้อมูล 1 ปี
            stock = yf.Ticker(symbol)
            data = stock.history(period=f"{days}d")

            if len(data) > 0:
                print(f"✅ ดึงข้อมูล {symbol} สำเร็จ: {len(data)} วัน")
                return data
            else:
                print(f"❌ ไม่มีข้อมูล {symbol}")
                return None

        except Exception as e:
            print(f"❌ ข้อผิดพลาด {symbol}: {e}")
            return None

    def calculate_indicators(self, data):
        """คำนวณตัวชี้วัดทางเทคนิค"""
        df = data.copy()

        # ราคาเฉลี่ย 20 วัน
        df['MA20'] = df['Close'].rolling(window=20).mean()

        # EMA (Exponential Moving Average)
        df['EMA12'] = df['Close'].ewm(span=12).mean()
        df['EMA26'] = df['Close'].ewm(span=26).mean()

        # MACD
        df['MACD'] = df['EMA12'] - df['EMA26']
        df['MACD_Signal'] = df['MACD'].ewm(span=9).mean()

        # RSI (Relative Strength Index)
        df['RSI'] = ta.momentum.rsi(df['Close'], window=14)

        # Bollinger Bands
        df['BB_Middle'] = df['Close'].rolling(window=20).mean()
        bb_std = df['Close'].rolling(window=20).std()
        df['BB_Upper'] = df['BB_Middle'] + (bb_std * 2)
        df['BB_Lower'] = df['BB_Middle'] - (bb_std * 2)

        # การเปลี่ยนแปลงราคา
        df['Price_Change'] = df['Close'].pct_change()

        # เป้าหมาย: ราคาวันถัดไป
        df['Next_Price'] = df['Close'].shift(-1)
        df['Target'] = (df['Next_Price'] / df['Close']) - 1

        # ลบข้อมูลที่ไม่ครบ
        df = df.dropna()

        return df

    def predict_direction(self, data):
        """ทำนายทิศทางราคา"""
        # เตรียมข้อมูลสำหรับ AI
        features = ['Close', 'Volume', 'MA20', 'EMA12', 'EMA26',
                   'MACD', 'MACD_Signal', 'RSI', 'BB_Upper', 'BB_Lower',
                   'Price_Change']

        X = data[features]
        y = data['Target']

        # แบ่งข้อมูล train/test
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        # ใช้ Random Forest (ง่ายกว่า TFT)
        model = RandomForestRegressor(n_estimators=100, random_state=42)
        model.fit(X_train, y_train)

        # ทำนายวันล่าสุด
        last_features = X.iloc[-1:].values
        prediction = model.predict(last_features)[0]

        # แปลงเป็นทิศทาง
        if prediction > 0.01:  # ขึ้น > 1%
            direction = "UP"
            emoji = "🔼"
            color = "green"
        elif prediction < -0.01:  # ลง > 1%
            direction = "DOWN"
            emoji = "🔽"
            color = "red"
        else:  # ไม่เปลี่ยนแปลง
            direction = "SIDEWAYS"
            emoji = "➡️"
            color = "gray"

        confidence = abs(prediction) * 100
        confidence = min(confidence, 100)  # จำกัดไม่เกิน 100%

        return {
            'prediction': prediction,
            'direction': direction,
            'emoji': emoji,
            'color': color,
            'confidence': confidence
        }

    def create_chart(self, symbol, data, prediction):
        """สร้างกราฟแสดงผล"""
        # ข้อมูล 30 วันล่าสุด
        recent_data = data.tail(30)

        fig = go.Figure()

        # กราฟเทียน
        fig.add_trace(go.Candlestick(
            x=recent_data.index,
            open=recent_data['Open'],
            high=recent_data['High'],
            low=recent_data['Low'],
            close=recent_data['Close'],
            name=f'{symbol} Price'
        ))

        # เส้นเฉลี่ย 20 วัน
        fig.add_trace(go.Scatter(
            x=recent_data.index,
            y=recent_data['MA20'],
            line=dict(color='blue', width=2),
            name='MA20'
        ))

        # Bollinger Bands
        fig.add_trace(go.Scatter(
            x=recent_data.index,
            y=recent_data['BB_Upper'],
            line=dict(color='gray', dash='dash'),
            name='BB Upper'
        ))

        fig.add_trace(go.Scatter(
            x=recent_data.index,
            y=recent_data['BB_Lower'],
            line=dict(color='gray', dash='dash'),
            name='BB Lower',
            fill='tonexty',
            fillcolor='rgba(128,128,128,0.1)'
        ))

        # ลูกศรทำนาย
        last_price = recent_data['Close'].iloc[-1]
        last_date = recent_data.index[-1]

        arrow_symbol = 'triangle-up' if prediction['direction'] == 'UP' else 'triangle-down'

        fig.add_trace(go.Scatter(
            x=[last_date],
            y=[last_price],
            mode='markers',
            marker=dict(
                symbol=arrow_symbol,
                size=25,
                color=prediction['color'],
                line=dict(width=3, color='white')
            ),
            name=f"ทำนาย: {prediction['direction']}",
            text=f"ทิศทาง: {prediction['direction']}<br>ความมั่นใจ: {prediction['confidence']:.1f}%",
            hovertemplate='%{text}<extra></extra>'
        ))

        # ตั้งค่ากราฟ
        fig.update_layout(
            title=f"{symbol} - การวิเคราะห์และทำนาย {prediction['emoji']}",
            xaxis_title="วันที่",
            yaxis_title="ราคา ($)",
            xaxis_rangeslider_visible=False,
            height=600,
            template='plotly_white'
        )

        return fig

    def save_alert_to_file(self, message):
        """บันทึกการแจ้งเตือนลงไฟล์"""
        try:
            filename = f"stock_alerts_{datetime.now().strftime('%Y%m%d')}.txt"
            with open(filename, 'a', encoding='utf-8') as f:
                f.write(f"\n{datetime.now().strftime('%H:%M:%S')} - {message}\n")
                f.write("-" * 50 + "\n")
            print(f"✅ บันทึกการแจ้งเตือนในไฟล์: {filename}")
            return True
        except Exception as e:
            print(f"❌ ไม่สามารถบันทึกไฟล์ได้: {e}")
            return False

    def show_desktop_alert(self, message):
        """แสดงการแจ้งเตือนบนหน้าจอ"""
        print("\n" + "🔔" * 20)
        print("📢 การแจ้งเตือนหุ้น")
        print("🔔" * 20)
        print(message)
        print("🔔" * 20)

        # บันทึกลงไฟล์ด้วย
        self.save_alert_to_file(message)

        return True

    def analyze_all_stocks(self):
        """วิเคราะห์หุ้นทั้งหมด"""
        print("🚀 เริ่มวิเคราะห์หุ้น...")
        print("=" * 40)

        results = {}
        line_messages = []

        for symbol in self.stocks:
            print(f"\n📊 กำลังวิเคราะห์ {symbol}...")

            # ดึงข้อมูล
            raw_data = self.get_stock_data(symbol)
            if raw_data is None:
                continue

            # คำนวณตัวชี้วัด
            data = self.calculate_indicators(raw_data)

            # ทำนาย
            prediction = self.predict_direction(data)

            # เก็บผลลัพธ์
            current_price = data['Close'].iloc[-1]
            results[symbol] = {
                'data': data,
                'prediction': prediction,
                'current_price': current_price
            }

            # สร้างกราฟ
            chart = self.create_chart(symbol, data, prediction)
            chart.show()

            # เตรียมข้อความ LINE
            message = f"{prediction['emoji']} {symbol}: {prediction['direction']}\n"
            message += f"💰 ราคา: ${current_price:.2f}\n"
            message += f"📈 ความมั่นใจ: {prediction['confidence']:.1f}%"
            line_messages.append(message)

            print(f"✅ {symbol}: {prediction['direction']} (มั่นใจ {prediction['confidence']:.1f}%)")

        # แสดงการแจ้งเตือนบนหน้าจอ
        if line_messages:
            full_message = "🤖 สัญญาณหุ้นวันนี้\n" + "=" * 25 + "\n\n"
            full_message += "\n\n".join(line_messages)
            full_message += f"\n\n⏰ {datetime.now().strftime('%d/%m/%Y %H:%M')}"

            # แสดงบนหน้าจอและบันทึกไฟล์
            self.show_desktop_alert(full_message)

        return results

    def quick_check(self, symbol):
        """เช็คหุ้นตัวเดียวแบบเร็ว"""
        print(f"⚡ เช็คด่วน {symbol}...")

        data = self.get_stock_data(symbol, days=180)  # 6 เดือน
        if data is None:
            return None

        processed_data = self.calculate_indicators(data)
        prediction = self.predict_direction(processed_data)
        current_price = processed_data['Close'].iloc[-1]

        print(f"📈 {symbol}: ${current_price:.2f}")
        print(f"{prediction['emoji']} ทำนาย: {prediction['direction']}")
        print(f"🎯 ความมั่นใจ: {prediction['confidence']:.1f}%")

        return {
            'symbol': symbol,
            'price': current_price,
            'prediction': prediction
        }

In [ ]:
if __name__ == "__main__":
    # สร้างตัววิเคราะห์
    predictor = SimpleStockPredictor()

    print("🎯 เลือกการใช้งาน:")
    print("1. วิเคราะห์หุ้นทั้งหมด (AAPL, TSLA, MSFT, NVDA, AMZN)")
    print("2. เช็คหุ้นตัวเดียวแบบเร็ว")
    print("3. ตั้งการแจ้งเตือนอัตโนมัติ")

    choice = input("เลือก (1, 2, หรือ 3): ")

    if choice == "1":
        # วิเคราะห์หุ้นทั้งหมด
        results = predictor.analyze_all_stocks()

        print("\n" + "=" * 50)
        print("📋 สรุปผลการวิเคราะห์:")
        print("=" * 50)

        for symbol, result in results.items():
            pred = result['prediction']
            price = result['current_price']
            print(f"{pred['emoji']} {symbol}: ${price:.2f} -> {pred['direction']} ({pred['confidence']:.1f}%)")

    elif choice == "2":
        # เช็คหุ้นตัวเดียว
        symbol = input("ใส่ชื่อหุ้น (เช่น AAPL): ").upper()
        result = predictor.quick_check(symbol)

        if result:
            # แสดงกราฟ
            data = predictor.get_stock_data(symbol)
            processed_data = predictor.calculate_indicators(data)
            chart = predictor.create_chart(symbol, processed_data, result['prediction'])
            chart.show()

    elif choice == "3":
        # ตั้งการแจ้งเตือนอัตโนมัติ
        print("🔔 ระบบแจ้งเตือนอัตโนมัติ")
        print("จะวิเคราะห์และแสดงผลทุก 30 นาที")
        print("กด Ctrl+C เพื่อหยุด")

        import time
        try:
            while True:
                print(f"\n⏰ {datetime.now().strftime('%H:%M:%S')} - กำลังวิเคราะห์...")
                predictor.analyze_all_stocks()
                print("💤 รอ 30 นาที...")
                time.sleep(1800)  # รอ 30 นาที
        except KeyboardInterrupt:
            print("\n👋 หยุดการแจ้งเตือนอัตโนมัติแล้ว")

    else:
        print("❌ เลือกไม่ถูกต้อง")

print("\n🎉 เสร็จสิ้น! ขอบคุณที่ใช้งาน")

🎯 เลือกการใช้งาน:
1. วิเคราะห์หุ้นทั้งหมด (AAPL, TSLA, MSFT, NVDA, AMZN)
2. เช็คหุ้นตัวเดียวแบบเร็ว
3. ตั้งการแจ้งเตือนอัตโนมัติ
เลือก (1, 2, หรือ 3): 1
🚀 เริ่มวิเคราะห์หุ้น...

📊 กำลังวิเคราะห์ AAPL...
📥 กำลังดึงข้อมูล AAPL...
✅ ดึงข้อมูล AAPL สำเร็จ: 365 วัน


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


✅ AAPL: UP (มั่นใจ 1.0%)

📊 กำลังวิเคราะห์ TSLA...
📥 กำลังดึงข้อมูล TSLA...
✅ ดึงข้อมูล TSLA สำเร็จ: 365 วัน


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning:

X does not have valid feature names, but RandomForestRegressor was fitted with feature names



✅ TSLA: DOWN (มั่นใจ 4.1%)

📊 กำลังวิเคราะห์ MSFT...
📥 กำลังดึงข้อมูล MSFT...
✅ ดึงข้อมูล MSFT สำเร็จ: 365 วัน


...

✅ MSFT: SIDEWAYS (มั่นใจ 0.5%)

📊 กำลังวิเคราะห์ NVDA...
📥 กำลังดึงข้อมูล NVDA...
✅ ดึงข้อมูล NVDA สำเร็จ: 365 วัน


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning:

X does not have valid feature names, but RandomForestRegressor was fitted with feature names



✅ NVDA: DOWN (มั่นใจ 2.3%)

📊 กำลังวิเคราะห์ AMZN...
📥 กำลังดึงข้อมูล AMZN...
✅ ดึงข้อมูล AMZN สำเร็จ: 365 วัน


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning:

X does not have valid feature names, but RandomForestRegressor was fitted with feature names



✅ AMZN: SIDEWAYS (มั่นใจ 0.5%)

🔔🔔🔔🔔🔔🔔🔔🔔🔔🔔🔔🔔🔔🔔🔔🔔🔔🔔🔔🔔
📢 การแจ้งเตือนหุ้น
🔔🔔🔔🔔🔔🔔🔔🔔🔔🔔🔔🔔🔔🔔🔔🔔🔔🔔🔔🔔
🤖 สัญญาณหุ้นวันนี้

🔼 AAPL: UP
💰 ราคา: $205.17
📈 ความมั่นใจ: 1.0%

🔽 TSLA: DOWN
💰 ราคา: $317.66
📈 ความมั่นใจ: 4.1%

➡️ MSFT: SIDEWAYS
💰 ราคา: $497.41
📈 ความมั่นใจ: 0.5%

🔽 NVDA: DOWN
💰 ราคา: $157.99
📈 ความมั่นใจ: 2.3%

➡️ AMZN: SIDEWAYS
💰 ราคา: $219.39
📈 ความมั่นใจ: 0.5%

⏰ 02/07/2025 02:51
🔔🔔🔔🔔🔔🔔🔔🔔🔔🔔🔔🔔🔔🔔🔔🔔🔔🔔🔔🔔
✅ บันทึกการแจ้งเตือนในไฟล์: stock_alerts_20250702.txt

📋 สรุปผลการวิเคราะห์:
🔼 AAPL: $205.17 -> UP (1.0%)
🔽 TSLA: $317.66 -> DOWN (4.1%)
➡️ MSFT: $497.41 -> SIDEWAYS (0.5%)
🔽 NVDA: $157.99 -> DOWN (2.3%)
➡️ AMZN: $219.39 -> SIDEWAYS (0.5%)

🎉 เสร็จสิ้น! ขอบคุณที่ใช้งาน
